** Разработка и тестирование приложения для прогнозирования будущих событий с использованием машинного обучения в промышленности.**

1. Сначало загрузим набор данных и выполним некую первоначальную подшготовку: переименнуем столбцы и изменим значения внутри столбцов, чтобы они были согласованы по всему набору данных.

2. Разделим данные на обучающие, проверочные и тестовые, чтобы иметь возможность проверять наши модели.

3. В рамках первоначального анализа данных мы рассмотрим важность признаков, чтобы определить, какие из них важны в данных.

4. Преобразуем категориальные переменные в числовые и, таким образом, сможем использовать их в модели.

5. Обучим модель логистической регресии.

                   Scikit-learn:
                  для разеления набора данных на обучающий и тестовый;
                  кодирования категориальных переменных;
                  обучения логистической регресии.

**DF содержит **
1. услуги клиентов - телефон; несколько линий; Интернет; техническая поддержка и дополнительные услуги, такие как онлай безопасность, резервное копирование, защита устройств и потоковое ТВ

2. информация об учетной записи - как долго остовались клиентами, тип контракта, способ оплаты;

3. сборы - сколько клиент заплатил за последний месяц и  в обще сложности;

4. демографическая информация - пол, возраст, имеются ли иждивенцы или партнер;

5. отток - да/нет, покинул ли клиент компанию в течении последнего месяца

# Часть 1 Обработка данных

1. Подготовка исходных данных

In [1]:
import pandas as pd
import numpy as np


import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
# читаем данные
df = pd.read_csv('https://raw.githubusercontent.com/architect-ai2023/final-attestation/main/telcom.csv')

In [5]:
# количество строк
len(df)

7043

In [6]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
# Транспонируем df
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


* CustomerID - идентификатор клиета
* Gender - мужчина/женщина
* SeniorCitizen - является ли клиент пожилым (0/1)
* Partner - проживает ли он с партнером (да/нет)
* Dependents - имеются ли у него иждивенцы (да/нет)
* Tenure - количество месяцев с момента начала действия контракта
* PhoneService - есть ли у него телефонная служба (да/нет)
* MultipleLines - имеется ли у него несколько телефонных линий (да/нет/нет телефонной связи)
* InternetService - тип интернет-сервиса (нет/кабель/оптпволокно)
* OnlineSecurity - подключена ли онлайн-безопасность (да/нет/нет Интенета)
* OnlineBackup - подключен ли онлайн-сервис резервног копирования (да/нет/нет Интенета)
* DeviceProtection - подключен ли сервис защиты устройств (да/нет/нет Интенета)
* TechSupport - есть ли у клиента техническая поддеожка (да/нет/нет Интенета)
* StreamingTV - подключен ли сервис потоковой передачи ТВ (да/нет/нет Интенета)
* StreamingMovies - подключен ли сервис потоковой передачи фильмов (да/нет/нет Интенета)
* Contract - тип контракта (ежемесячный/годовой/двухлетний)
* PaperlessBilling - подключено ли электронное выставление счетов (да/нет)
* PaymentMethod - способ оплаты (электронный чек, чек по почте, банковский перевод, кредитная карта)
* MonthlyCharges - сумма, взимаемая ежемесячно (число)
* TotalCharges - общая сумма начислений (число)
* Churn - расторг ли клиент контракт (да/нет)

Саммый интересный для нас пункт - это Churn. В качестве целевой переменной для нашей модели мы хотим научиться прогнозировать именно его. Он принемает два значения: да, если клиет расторг контракт, и нет, если он этого не сделал.

In [8]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [9]:
# заменим все не чисовые значения на NaN
total_charges = pd.to_numeric(df.TotalCharges, errors='coerce')

In [10]:
# в TotalCharges встечаются пробелы
df[total_charges.isnull()][['customerID', 'TotalCharges']]

,customerID,TotalCharges
488,4472-LVYGI,
753,3115-CZMZD,
936,5709-LVOEQ,
1082,4367-NUYAO,
1340,1371-DWPAZ,
3331,7644-OMVMY,
3826,3213-VVOLG,
4380,2520-SGTTA,
5218,2923-ARZLG,
6670,4075-WKNIU,


In [11]:
# установим недостающие значения равные 0
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')
df.TotalCharges = df.TotalCharges.fillna(0)

In [12]:
from IPython.utils.text import string
# приведем все к нижниму регистру и заменим пробелы символом подчеркивания
df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
  df[col] = df[col].str.lower().str.replace(' ', '_')

In [13]:
#  в churn данные преобразуем в числа
df.churn = (df.churn == 'yes').astype(int)
df.churn.head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int64

In [14]:
# Отделим часть данных для тестирования
from sklearn.model_selection import train_test_split

In [15]:
# функция train_test_split принимает датафрейм df и создает два новых датафрейма:
# df_train_full и df_test путем перетасовки исходного набора данных с последующем
# разделением на тестовый набор 20% данных, а обочающий 80%
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [16]:
# посмотрим что получилось
df_train_full.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
1814,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
5946,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
3881,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
2389,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
3676,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [17]:
# df_train_full разделяет "полный набор данных" на 2 части на "полный обучающий" и "тестовый"
# разделим "полный обучающий" на "обучающий" и "проверочный"
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

# принимает столбец с целевой переменной churn и сохроняет его за пределами датафрейма
y_train = df_train.churn.values
y_val = df_val.churn.values

# удаляем столбцы churn из обоих датафреймов для гарантии того, что мы случайно
# не используем переменную churm в качестве признака при обучении
del df_train['churn']
del df_val['churn']

2. Исследвательский анализ данных

In [18]:
# проверка данных для того, нужно ли нам выполнять какую либо дополнительную обработку null
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [19]:
# проверим распределение значений в целевой переменной
df_train_full.churn.value_counts()

0    4113
1    1521
Name: churn, dtype: int64

Первый столбец - значение целевой переменной, а втрой -количество,
как мы видим большинство клиентов не расторгали договор. Мы знаем, что
отток составил 1521 из 5634 поэтому пропорция такова
                  1521/5634=0,27
Как мы видим из обучающего набора данных, примерно 27% клиентов перестали пользоваться нашими услугами. "Коэффициент оттока"

In [20]:
# проверим
global_mean = df_train_full.churn.mean()
round(global_mean, 3)

0.27

In [21]:
df_train_full.head().T

,1814,5946,3881,2389,3676
customerid,5442-pptjy,6261-rcvns,2176-osjuv,6161-erdgd,2364-ufrom
gender,male,female,male,male,male
seniorcitizen,0,0,0,0,0
partner,yes,no,yes,yes,no
dependents,yes,no,no,yes,no
tenure,12,42,71,71,30
phoneservice,yes,yes,yes,yes,yes
multiplelines,no,no,yes,yes,no
internetservice,no,dsl,dsl,dsl,dsl
onlinesecurity,no_internet_service,yes,yes,yes,yes


In [22]:
# разделим на категориальные и числовые
from pandas.core.arrays import categorical
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup',
               'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [23]:
# узнаем сколько уникальных значений содержит каждая переменная
df_train_full[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

3. Важность признака
знание того, как другие переменные влияют на целевую переменную (отток), - ключ к пониманию данных и построению хорошей модели.
Аннализ важности признаков - позволяет выяснить, какие переменные полезны для модели. Поможет выяснить, Что вызывает отток клиентов? и Каковы характеристики людей, которые уходят?
 Измерять важность признаков можно раными способами

In [24]:
# Расмотрим кеффициент оттока для каждой переменной
# Если разница между глобальным коэффициентом оттока не велика то и значение не очень важно при прогнозировании оттока

# global_mean
global_mean = df_train_full.churn.mean()
round(global_mean, 3)
print('global_mean: ', round(global_mean, 3))

# Сначало проверим переменную gender == 'female' женсого пола
female_mean = df_train_full[df_train_full.gender == 'female'].churn.mean()
print('gender == female: ', round(female_mean, 3))

# переменную gender == 'male' мужского пола
male_mean = df_train_full[df_train_full.gender == 'male'].churn.mean()
print('gender == male: ', round(male_mean, 3))

global_mean:  0.27
gender == female:  0.277
gender == male:  0.263


Видим что разница между группами мужчина и женщина не велика, знание пола клиента нам не поможет определить, расторгнет ли он договор.

In [25]:
# Посмотрим переменную partner
global_mean = df_train_full.churn.mean()
round(global_mean, 3)
print('global_mean: ', round(global_mean, 3))

partner_yes = df_train_full[df_train_full.partner == 'yes'].churn.mean()
print('partner == yes: ', round(partner_yes, 3))

partner_no = df_train_full[df_train_full.partner == 'no'].churn.mean()
print('partner == no: ', round(partner_no, 3))

global_mean:  0.27
partner == yes:  0.205
partner == no:  0.33


Коэфициент оттока с партнером значительно ниже, чем среди людей без партнера, 20,5% против 33% , что указывает на то, что переменная partner ролезна для прогнозирования оттока.

**Коэффициент риска**
 В ститистике соотношение между вероятностями в разных группах называется коэфициентом риска, где риск относится к риску возникновения эффекта. В нашем случае эффект - это отток, так что риск оттока выглядит так

       риск = групповой коэффициент / глобальный коэффициент
       например, для gender == female риск оттока составляет 1,02
       риск = 27,7% / 27% = 1,02
  
Риск - это число от нуля до бесконечности. У него есть полезная интерпритация, которая показывает вероятность того, насколько элементы группы испытают эффект(отток) по сравнению со всей совокупностью элементов.

In [26]:
# Определим риски для всех категориальных признаков
from IPython.display import display

for col in categorical:
  df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
  df_group['diff'] = df_group['mean'] - global_mean
  df_group['rate'] = df_group['mean'] / global_mean
  display(df_group)

,mean,diff,rate
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,rate
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,rate
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,rate
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,rate
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,rate
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,rate
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,rate
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,rate
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,rate
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


Взаимная информация
определим наиболее важные признаки

In [27]:
from sklearn.metrics import mutual_info_score

def calculate_mi(series):
  return mutual_info_score(series, df_train_full.churn)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name="MI")
df_mi

,MI
contract,0.098320
onlinesecurity,0.063085
techsupport,0.061032
internetservice,0.055868
onlinebackup,0.046923
deviceprotection,0.043453
paymentmethod,0.043210
streamingtv,0.031853
streamingmovies,0.031581
paperlessbilling,0.017589


Как мы видим, contract, onlinesecurity, techsupport относятся к числу наиболее важных признаков.

Коэффициент корреляции

In [28]:
df_train_full[numerical].corrwith(df_train_full.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

# Часть 2 Конструирование признаков
прежде чем приступать к обучению нам нужно выполнить этап конструирования признаков: преобразовать все категориальные переменные в числовые признаки.

In [29]:
# преобразуем датафрейм в в список словарей
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [30]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [31]:
X_train = dv.transform(train_dict)
X_train[0]

array([0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 8.6100e+01, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 7.1000e+01, 6.0459e+03])

In [32]:
dv.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen',
       'streamingmovies=no', 'streamingmovies=no_internet_service',

# 3 Машинное обучение для классификации

Логистическая регрессия

In [33]:
# Сигмоидальная функция
import math

def sigmoid(score):
  return 1 / (1 + math.exp(-score))

Обучение логистической регрессии

In [34]:
# Импортируем модель
from sklearn.linear_model import LogisticRegression

# обучим модель
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [35]:
# преобразуем датафрейм в список словарей и предадим его DictVectorizer
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [36]:
# применим ее к модели
y_pred = model.predict_proba(X_val)
y_pred

array([[0.76508784, 0.23491216],
       [0.73113015, 0.26886985],
       [0.68054704, 0.31945296],
       ...,
       [0.94274614, 0.05725386],
       [0.38476895, 0.61523105],
       [0.93872763, 0.06127237]])

Результатом predict_proba служит духмерный массив. Первый столбец содержит вероятность того, что цель отрицательна(отток отсутсвует), а второй - вероятность того, что она положительна(отток).
Столбец с лева - клиент останется, с права - клиент уйдет.
Нам достаточно иметь только один столбец

In [37]:
# Клиенты которые уйдут
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.23491216, 0.26886985, 0.31945296, ..., 0.05725386, 0.61523105,
       0.06127237])

In [38]:
# уточним прогноз
churn = y_pred >= 0.5

Следующий шаг оценка качества прогноза

In [39]:
# Такой показатель качества называется достоверностью
(y_val == churn).mean()

0.8016129032258065

Интерпритация модели

In [40]:
# Посмотрим какой признак связан с каждым вессом
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'contract=month-to-month': 0.563,
 'contract=one_year': -0.086,
 'contract=two_year': -0.599,
 'dependents=no': -0.03,
 'dependents=yes': -0.092,
 'deviceprotection=no': 0.1,
 'deviceprotection=no_internet_service': -0.116,
 'deviceprotection=yes': -0.106,
 'gender=female': -0.027,
 'gender=male': -0.095,
 'internetservice=dsl': -0.323,
 'internetservice=fiber_optic': 0.317,
 'internetservice=no': -0.116,
 'monthlycharges': 0.001,
 'multiplelines=no': -0.168,
 'multiplelines=no_phone_service': 0.127,
 'multiplelines=yes': -0.081,
 'onlinebackup=no': 0.136,
 'onlinebackup=no_internet_service': -0.116,
 'onlinebackup=yes': -0.142,
 'onlinesecurity=no': 0.258,
 'onlinesecurity=no_internet_service': -0.116,
 'onlinesecurity=yes': -0.264,
 'paperlessbilling=no': -0.213,
 'paperlessbilling=yes': 0.091,
 'partner=no': -0.048,
 'partner=yes': -0.074,
 'paymentmethod=bank_transfer_(automatic)': -0.027,
 'paymentmethod=credit_card_(automatic)': -0.136,
 'paymentmethod=electronic_check': 0.175,


Использование модели

In [41]:
customer = {
    'customerid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen': '0',
    'partner': 'no',
    'dependents': 'no',
    'tenure': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75
}

In [42]:
# Теперь мы можем использовать нашу модель и узнать, подвержен ли данный клиент оттоку
X_test = dv.transform([customer])
X_test

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 7.98500e+01, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00,
        1.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 4.10000e+01, 3.32075e+03]])

In [43]:
# Полученую матрицу отправляем в обученую модель
# На выходе получаем матрицу с прогнозами, для каждго клиента она выводит 2 числа
# [останется, вероятность оттока]
model.predict_proba(X_test)

array([[0.92667889, 0.07332111]])

In [44]:
# только отток
model.predict_proba(X_test)[0, 1]

0.07332111084949638